<a href="https://colab.research.google.com/github/nicosesma/otc_strategies/blob/main/DCA_Backtester_script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

class DCABacktestingEngine:
    """
    A backtesting engine to simulate a Dollar-Cost Averaging (DCA) trading strategy.

    This class simulates trades on historical data based on user-defined parameters
    for entry, averaging, and take-profit levels.
    """

    def __init__(self,
                 base_order_size,
                 averaging_order_size_multiplier,
                 averaging_order_step,
                 max_average_orders,
                 take_profits_percent):
        """
        Initializes the backtesting engine with DCA strategy parameters.

        Args:
            base_order_size (float): The size of the initial base order.
            averaging_order_size_multiplier (float): Multiplier for each subsequent averaging order.
            averaging_order_step (float): The percentage drop from the last order price to trigger a new order.
            max_average_orders (int): The maximum number of averaging orders per trade.
            take_profits_percent (list): A list of take-profit percentages (e.g., [1.0, 2.0, 3.0]).
        """
        self.base_order_size = base_order_size
        self.averaging_order_size_multiplier = averaging_order_size_multiplier
        self.averaging_order_step = averaging_order_step / 100.0  # Convert to decimal
        self.max_average_orders = max_average_orders
        self.take_profits_percent = [tp / 100.0 for tp in take_profits_percent] # Convert to decimals

    def run(self, historical_data):
        """
        Runs the backtest simulation on the provided historical data.

        This method simulates a single, complete DCA trade at a time. It will not
        open a new trade until the current one is fully closed.

        Args:
            historical_data (pd.DataFrame): A dataframe with 'close' prices.

        Returns:
            dict: A dictionary with key performance metrics.
        """
        if historical_data.empty:
            print("Warning: Input data is empty.")
            return {"total_profit_loss": 0, "number_of_trades": 0, "roi": 0}

        # --- Initialize trading variables ---
        initial_capital = 1000.0  # Assumed initial capital
        open_position = False
        total_profit_loss = 0
        trade_count = 0

        # Variables to track the current trade
        trade = {
            'entry_prices': [],
            'entry_volume': [],
            'current_orders': 0,
            'avg_price': 0.0
        }

        # --- Main backtesting loop ---
        for i in range(len(historical_data)):
            current_close = historical_data['close'].iloc[i]

            # 1. Entry Logic: Open a new trade if none exists
            if not open_position:
                # For simplicity, we'll start a new trade when there's no open position
                # and we have enough capital. In a real bot, you'd use a signal here.
                trade_count += 1
                open_position = True
                order_size = self.base_order_size

                trade = {
                    'entry_prices': [current_close],
                    'entry_volume': [order_size],
                    'current_orders': 1,
                }
                trade['avg_price'] = current_close
                print(f"[{historical_data.index[i]}] BUY BASE ORDER. Price: {current_close:.2f}")

            # 2. Averaging Down Logic: Add to the position if price drops
            else:
                last_order_price = trade['entry_prices'][-1]
                price_drop = (last_order_price - current_close) / last_order_price

                # Check if a new averaging order should be placed
                if (price_drop >= self.averaging_order_step and
                    trade['current_orders'] <= self.max_average_orders):

                    order_size = self.base_order_size * (self.averaging_order_size_multiplier ** (trade['current_orders'] - 1))

                    trade['entry_prices'].append(current_close)
                    trade['entry_volume'].append(order_size)
                    trade['current_orders'] += 1

                    # Update average price
                    total_cost = sum(p * v for p, v in zip(trade['entry_prices'], trade['entry_volume']))
                    total_volume = sum(trade['entry_volume'])
                    trade['avg_price'] = total_cost / total_volume

                    print(f"[{historical_data.index[i]}] BUY AVERAGING ORDER. Price: {current_close:.2f} | Avg Price: {trade['avg_price']:.2f}")

            # 3. Take Profit Logic: Check if any TP level is hit
            if open_position:
                for tp_percent in self.take_profits_percent:
                    if (current_close >= trade['avg_price'] * (1 + tp_percent)):
                        # Trade closed with profit
                        profit = (current_close - trade['avg_price']) * sum(trade['entry_volume'])
                        total_profit_loss += profit
                        open_position = False # Reset for a new trade
                        print(f"[{historical_data.index[i]}] SELL TAKE PROFIT. Price: {current_close:.2f} | PnL: {profit:.2f}")
                        break # Exit the TP loop once one level is hit

        final_equity = initial_capital + total_profit_loss
        roi = (total_profit_loss / initial_capital) * 100 if initial_capital > 0 else 0

        # Return performance metrics
        return {
            "total_profit_loss": total_profit_loss,
            "number_of_trades": trade_count,
            "final_equity": final_equity,
            "roi": roi
        }

# --- Example Usage ---
if __name__ == '__main__':
    # Create mock historical data
    data_points = 500
    dates = pd.date_range(start='2024-01-01', periods=data_points)
    # Generate mock price data with some dips to trigger DCA
    mock_prices = 100 + np.random.randn(data_points).cumsum() * 0.5 + np.sin(np.linspace(0, 15, data_points)) * 5
    mock_df = pd.DataFrame({'close': mock_prices}, index=dates)

    # Instantiate and run the backtester with the user's parameters
    backtester = DCABacktestingEngine(
        base_order_size=10.0,  # Base Order
        averaging_order_size_multiplier=1.5,
        averaging_order_step=1.0,  # Price Deviation
        max_average_orders=3,  # Max Average Order Per Trade
        take_profits_percent=[1.0, 2.0, 3.0, 4.0] # Take Profit 1-4
    )
    performance = backtester.run(mock_df)

    print("\n--- Backtest Summary ---")
    print(f"Initial Capital: $1000.00")
    print(f"Final Equity: ${performance['final_equity']:.2f}")
    print(f"Total Profit/Loss: ${performance['total_profit_loss']:.2f}")
    print(f"Number of Trades: {performance['number_of_trades']}")
    print(f"ROI: {performance['roi']:.2f}%")